In [1]:
import pandas as pd
from pathlib import Path
import pymc as pm
import plotly.express as px
import numpy as np
import pandas as pd

from estival.model import BayesianCompartmentalModel
import estival.priors as esp
import estival.targets as est
from estival.wrappers import pymc as epm
from tbdynamics import model
from tbdynamics.utils import round_sigfig
from tbdynamics.inputs import fixed_parameters
import plotly.graph_objects as go

WARNING (pytensor.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
WARNING (pytensor.configdefaults): g++ not detected!  PyTensor will be unable to compile C-implementations and will default to Python. Performance may be severely degraded. To remove this warning, set PyTensor flags cxx to an empty string.
WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


## Define variables

In [2]:
PROJECT_PATH = Path().resolve()
DATA_PATH = PROJECT_PATH / 'data'
pd.options.plotting.backend = "plotly"

time_start = 1700.0
time_end = 2023.0
time_step = 0.1

compartments = [
    "susceptible",
    "early_latent",
    "late_latent",
    "infectious",
    "on_treatment",
    "recovered",
]

latent_compartments = [
    "early_latent",
    "late_latent",
]
infectious_compartments = [
    "infectious",
    "on_treatment",
]


age_strata = [0,5,15,35,50,70] 
matrix = np.ones((6, 6)) # Homo mixing

## Define Model

### Base model

In [3]:
tb_model = model.build_model(
    compartments,
    latent_compartments,
    infectious_compartments,
    age_strata,
    time_start,
    time_end,
    time_step,
    fixed_parameters,
    matrix
)

### Params and calibration targets

In [4]:
params = {
    "smear_positive_death_rate":0.364337776897486,
    "smear_negative_death_rate": 0.027588310343242016, 
    "smear_positive_self_recovery": 0.20344728302826143,
    "smear_negative_self_recovery": 0.22723824998716693,
}


priors = [
    esp.UniformPrior("start_population_size", (100000, 30000000)),
    esp.UniformPrior("contact_rate", (0, 10)),  # multiplied with eigenvalue
    esp.UniformPrior("rr_infection_latent", (0.2, 0.5)),
    esp.UniformPrior("rr_infection_recovered", (0.1, 0.5)),
    esp.UniformPrior("progression_multiplier", (1.0, 10.0)),
    esp.UniformPrior("seed_time", [1700.0, 1900.0]),
    esp.UniformPrior("seed_duration", [1.0, 5.0]),
    esp.UniformPrior("seed_num", [1.0, 1000.0]),
    esp.UniformPrior("smear_positive_death_rate", (0.335, 0.449)),
    esp.UniformPrior("smear_negative_death_rate", (0.017, 0.035)),
    esp.UniformPrior("smear_positive_self_recovery", (0.177, 0.288)),
    esp.UniformPrior("smear_negative_self_recovery", (0.073, 0.209)),

]

pop = pd.Series(
    {
        2009: 86025000,
        2019: 96484000,

    }
)

targets = [
    est.NormalTarget("total_population", pop, stdev=100000),
]

calibration_model = BayesianCompartmentalModel(tb_model, params, priors, targets)

### Running Calibration

In [11]:
with pm.Model() as pmc_model:
    start_params = {k: np.clip(v, *calibration_model.priors[k].bounds(0.99)) for k, v in params.items() if k in calibration_model.priors}
    variables = epm.use_model(calibration_model)
    map_params = pm.find_MAP(start=start_params, vars=variables, maxeval= 1000,include_transformed=False)
    map_params = {k: float(v) for k, v in map_params.items()}
    print('Best calibration parameters found:')
for i_param, param in enumerate(map_params):
    print(f'   {param}: {round_sigfig(map_params[param], 4)} (within bound {priors[i_param].bounds()}')


Best calibration parameters found:
   start_population_size: 814200.0 (within bound (100000.0, 30000000.0)
   contact_rate: 4.657 (within bound (0.0, 10.0)
   rr_infection_latent: 0.3397 (within bound (0.2, 0.5)
   rr_infection_recovered: 0.2681 (within bound (0.1, 0.5)
   progression_multiplier: 5.599 (within bound (1.0, 10.0)
   seed_time: 1783.0 (within bound (1700.0, 1900.0)
   seed_duration: 4.974 (within bound (1.0, 5.0)
   seed_num: 491.0 (within bound (1.0, 1000.0)
   smear_positive_death_rate: 0.4489 (within bound (0.335, 0.449)
   smear_negative_death_rate: 0.01815 (within bound (0.017, 0.035)
   smear_positive_self_recovery: 0.2 (within bound (0.177, 0.288)
   smear_negative_self_recovery: 0.2048 (within bound (0.073, 0.20900000000000002)


In [12]:
params.update(map_params)
tb_model.run(params)
derived_df_0 = tb_model.get_derived_outputs_df()

### Population output

In [13]:
fig2_1 = px.line(
    derived_df_0,
    x=derived_df_0.index,
    y="total_population",
)
fig2_2 = px.scatter(x= pop.index, y = pop.values)
fig2_2.update_traces(marker=dict(color="red"))
fig2_3 = go.Figure(
    data=fig2_1.data + fig2_2.data,
)
fig2_3.update_layout(
    title="Modelled vs Data", title_x=0.5, xaxis_title="Year", yaxis_title="Population"
)
fig2_3.show()

In [14]:
derived_df_0[[f'total_populationXage_{i}' for i in age_strata]].plot.area(title='Modelled populatation by age group')

In [15]:
derived_df_0['percentage_latent'].plot()

In [16]:
derived_df_0['prevalence_infectious'].plot()